In [1]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [9]:
round_ = '18'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-18'

In [10]:
day = datetime.now().strftime("%A").lower()
day

'thursday'

In [11]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [12]:
tmp = soup.find_all('script')

In [13]:
game_urls = []
for i in tmp[:9]:
    try:
        url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
        game_url = url_find.search(i.text)
        game_urls.append(game_url.group())
    except:
        continue
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/sydney-v-western-bulldogs-7470679',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/melbourne-v-brisbane-7470681',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/collingwood-v-fremantle-7470682',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/gold-coast-v-st-kilda-7470684',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/carlton-v-port-adelaide-7470685',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/geelong-v-essendon-7470687',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/adelaide-v-greater-western-sydney-7470688',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/north-melbourne-v-hawthorn-7470689',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/west-coast-v-richmond-7470691']

In [14]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [15]:
game_list

[[('sydney', 'western-bulldogs')],
 [('melbourne', 'brisbane')],
 [('collingwood', 'fremantle')],
 [('gold-coast', 'st-kilda')],
 [('carlton', 'port-adelaide')],
 [('geelong', 'essendon')],
 [('adelaide', 'greater-western-sydney')],
 [('north-melbourne', 'hawthorn')],
 [('west-coast', 'richmond')]]

In [16]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

[WDM] - Downloading: 100%|███████████████████| 8.29M/8.29M [00:00<00:00, 12.8MB/s]


In [17]:
disposals = ['To Get 15 or More Disposals',
            'To Get 20 or More Disposals',
            'To Get 25 or More Disposals',
            'To Get 30 or More Disposals',]
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [18]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
        
        
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,50):
            try:
                tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
            except:
                continue
            if tmp in disposals:

                dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

                dis.click()

                player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
                print(player)
                new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
                output.loc[len(output)] = new_row

                for j in range(1,50):
                    try:

                        tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                        print(tmp1)
                        new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                        output.loc[len(output)] = new_row
                    except:
                        continue
                
                dis.click()
                time.sleep(2)
            
    except:
        continue

['Nick Blakey', '1.06']
['Oliver Florent', '1.10']
['James Rowbottom', '1.16']
['Justin McInerney', '1.31']
['Isaac Heeney', '1.32']
['Braeden Campbell', '1.35']
['Robbie Fox', '1.37']
['Dane Rampe', '1.44']
['Harry Cunningham', '1.68']
['Tom Papley', '1.81']
['Hayden McLean', '2.25']
['Samuel Wicks', '2.30']
['Corey Warner', '2.45']
['Joel Amartey', '2.65']
['Will Hayward', '3.40']
['Tom McCartin', '3.80']
['Lance Franklin', '4.75']
['Tim English', '1.15']
['Ed Richards', '1.18']
['Bailey Williams (WB)', '1.50']
['Caleb Poulter', '1.79']
['Laitham Vandermeer', '2.30']
['Jamarra Ugle-Hagan', '2.35']
['Alex Keath', '2.90']
['Taylor Duryea', '2.90']
['Aaron Naughton', '3.30']
['Cody Weightman', '3.35']
['Luke Parker', '1.10']
['Errol Gulden', '1.20']
['Callum Mills', '1.23']
['Nick Blakey', '1.40']
['Oliver Florent', '1.45']
['James Rowbottom', '1.85']
['Justin McInerney', '2.25']
['Robbie Fox', '2.70']
['Braeden Campbell', '2.75']
['Isaac Heeney', '2.80']
['Dane Rampe', '3.05']
['Harry 

['Jordan Clark', 'Last 5: 26-20-17-22-14', '1.18']
['Matt Johnson', 'Last 5: N/A', '1.33']
['Sean Darcy', 'Last 5: 14-18-14-6-8', '1.48']
['Neil Erasmus', 'Last 5: 17-2-12-10-15', '1.49']
['Lachie Schultz', 'Last 5: 18-18-10-17-17', '1.59']
['Luke Jackson', 'Last 5: 10-8-17-17-23', '1.72']
['Sam Switkowski', 'Last 5: 13-11-16-15-11', '2.30']
['Michael Walters', 'Last 5: 8-15-6-3-8', '4.50']
['Tom Mitchell', 'Last 5: 24-30-27-30-30', '1.09']
['Josh Daicos', 'Last 5: 24-27-33-17-21', '1.16']
['Scott Pendlebury', 'Last 5: 21-27-24-26-21', '1.27']
['Jordan De Goey', 'Last 5: 23-19-35-27-31', '1.28']
['John Noble', 'Last 5: 28-30-14-20-17', '1.50']
['Jack Crisp', 'Last 5: 26-19-14-28-16', '1.71']
['Isaac Quaynor', 'Last 5: 27-24-15-16-18', '1.92']
['Taylor Adams', 'Last 5: 19-14-20-27-23', '1.95']
['Brayden Maynard', 'Last 5: 16-27-25-10-17', '2.30']
['Patrick Lipinski', 'Last 5: 16-23-20-15-19', '2.35']
['Darcy Moore', 'Last 5: 22-20-15-13-16', '2.70']
['Jeremy Howe', 'Last 5: 18-8-20-8-12

['Lachie Weller', 'Last 5: 19-14-12-18-28', '5.90']
['Darcy MacPherson', 'Last 5: 15-22-25-22-12', '6.00']
['Rory Atkins', 'Last 5: 26-14-7-21-17', '8.00']
['Matt Rowell', 'Last 5: 21-20-20-21-15', '9.25']
['Brayden Fiorini', 'Last 5: 19-21-35-16-17', '11.00']
['Wil Powell', 'Last 5: 14-27-19-18-17', '12.50']
['Brandon Ellis', 'Last 5: 17-18-16-14-12', '17.00']
['Ben Ainsworth', 'Last 5: 17-8-27-22-14', '23.00']
['Jack Sinclair', 'Last 5: 33-30-36-31-32', '1.65']
['Brad Crouch', 'Last 5: 27-26-25-33-33', '2.90']
['Jack Steele', 'Last 5: 31-22-21-21-24', '3.35']
['Nasiah Wanganeen-Milera', 'Last 5: 33-22-24-22-30', '4.00']
['Brad Hill', 'Last 5: 16-25-31-19-19', '8.00']
['Rowan Marshall', 'Last 5: 30-26-13-17-16', '8.00']
['Jade Gresham', 'Last 5: 24-21-11-16-18', '12.50']
['Mason Wood', 'Last 5: 21-24-24-18-16', '15.00']
['Zak Jones', 'Last 5: 20-14-17-4-18', '17.00']
['Callum Wilkie', 'Last 5: 23-16-15-17-23', '18.00']
['Marcus Windhager', 'Last 5: 19-20-12-20-15', '23.00']
['Blake Ac

['Sam Durham', 'Last 5: 17-14-15-17-15', '4.60']
['Jake Kelly', 'Last 5: 16-13-12-14-11', '6.00']
['Kyle Langford', 'Last 5: 12-16-7-11-14', '7.00']
['Tom Stewart', 'Last 5: 26-25-22-18-27', '1.92']
['Patrick Dangerfield', 'Last 5: 26-15-15-16-28', '1.95']
['Mitch Duncan', 'Last 5: 30-25-22-20-25', '2.20']
['Gryan Miers', 'Last 5: 32-27-15-13-20', '3.15']
['Isaac Smith', 'Last 5: 18-17-20-22-24', '4.30']
['Jack Bowes', 'Last 5: 25-3-12-5-3', '4.60']
['Max Holmes', 'Last 5: 26-8-17-24-13', '4.75']
['Zach Tuohy', 'Last 5: 18-28-18-24-20', '4.80']
['Tanner Bruhn', 'Last 5: 22-18-19-12-15', '7.00']
['Tom Atkins', 'Last 5: 25-18-19-12-19', '7.00']
['Mark Blicavs', 'Last 5: 18-16-19-13-17', '11.50']
['Zach Guthrie', 'Last 5: 21-10-12-26-16', '11.50']
['Jeremy Cameron', 'Last 5: 0-10-21-17-10', '12.00']
['Brad Close', 'Last 5: 16-16-24-9-18', '13.00']
['Darcy Parish', 'Last 5: 39-29-24-37-38', '1.10']
['Zach Merrett', 'Last 5: 39-31-29-18-34', '1.25']
['Nic Martin', 'Last 5: 28-16-26-30-21', 

['Tom Powell', 'Last 5: 12-16-8-16-12', '1.51']
['Tristan Xerri', 'Last 5: 19-5-5-14-14', '1.57']
['Charlie Lazzaro', 'Last 5: 15-5-1-8-15', '1.58']
['Flynn Perez', 'Last 5: 14-7-16-24-0', '1.89']
['Aidan Corr', 'Last 5: 12-15-12-8-14', '2.30']
['Eddie Ford', 'Last 5: 13-13-11-19-9', '2.30']
['Cooper Harvey', 'Last 5: N/A', '3.90']
['Dylan Moore', 'Last 5: 13-15-17-25-22', '1.10']
['Blake Hardwick', 'Last 5: 25-12-38-22-25', '1.13']
['Josh Ward', 'Last 5: 20-28-15-20-20', '1.13']
['Seamus Mitchell', 'Last 5: 20-17-24-21-20', '1.16']
['Jack Scrimshaw', 'Last 5: 17-18-31-19-19', '1.23']
['Connor Macdonald', 'Last 5: 23-15-13-28-20', '1.24']
['Josh Weddle', 'Last 5: 20-23-10-18-25', '1.24']
['Chad Wingard', 'Last 5: 16-18-8-12-17', '1.54']
['Mitch Lewis', 'Last 5: 11-9-15-11-20', '2.70']
['Sam Frost', 'Last 5: 7-9-9-8-15', '2.85']
['Tyler Brockman', 'Last 5: 11-9-17-15-8', '3.00']
['Luke Breust', 'Last 5: 10-8-11-15-11', '3.80']
['Luke Davies-Uniacke', 'Last 5: 28-22-29-25-20', '1.10']
['

['Dom Sheed', 'Last 5: 19-25-20-17-43', '1.99']
['Alex Witherden', 'Last 5: 22-29-17-15-28', '2.80']
['Jeremy McGovern', 'Last 5: 20-10-21-22-6', '3.05']
['Jayden Hunt', 'Last 5: 27-19-10-13-18', '4.00']
['Andrew Gaff', 'Last 5: 6-21-13-16-22', '5.25']
['Brady Hough', 'Last 5: 22-9-16-14-16', '11.50']
['Elijah Hewett', 'Last 5: 18-11-16-4-8', '11.50']
['Jack Petruccelle', 'Last 5: 17-16-22-17-15', '11.50']
['Tim Taranto', 'Last 5: 29-21-38-35-36', '1.22']
['Dustin Martin', 'Last 5: 29-35-27-19-13', '1.95']
['Jacob Hopper', 'Last 5: 22-22-10-32-21', '2.10']
['Shai Bolton', 'Last 5: 31-18-18-33-23', '2.20']
['Dion Prestia', 'Last 5: 22-25-27-23-29', '2.25']
['Daniel Rioli', 'Last 5: 18-18-25-20-17', '3.25']
['Nick Vlastuin', 'Last 5: 18-20-23-26-22', '3.90']
['Trent Cotchin', 'Last 5: 15-14-29-28-14', '5.25']
['Jack Ross', 'Last 5: 20-21-16-19-18', '6.25']
['Liam Baker', 'Last 5: 13-15-20-20-20', '6.75']
['Jack Graham', 'Last 5: 19-11-20-21-9', '8.00']
['Noah Balta', 'Last 5: 15-24-17-13

In [19]:
output

,odds,round,teams,disposals,Player
0,1.06,R18,sydney v western-bulldogs,To Get 15 or More Disposals,nick blakey
1,1.10,R18,sydney v western-bulldogs,To Get 15 or More Disposals,oliver florent
2,1.16,R18,sydney v western-bulldogs,To Get 15 or More Disposals,james rowbottom
3,1.31,R18,sydney v western-bulldogs,To Get 15 or More Disposals,justin mcinerney
4,1.32,R18,sydney v western-bulldogs,To Get 15 or More Disposals,isaac heeney
...,...,...,...,...,...
895,10.50,R18,west-coast v richmond,To Get 30 or More Disposals,trent cotchin
896,12.50,R18,west-coast v richmond,To Get 30 or More Disposals,nick vlastuin
897,21.00,R18,west-coast v richmond,To Get 30 or More Disposals,jack ross
898,26.00,R18,west-coast v richmond,To Get 30 or More Disposals,liam baker


In [20]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [21]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'north-melbourne':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [22]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [23]:
output

,odds,round,teams,disposals,Player,home,away
0,1.06,R18,sydney v western-bulldogs,To Get 15 or More Disposals,nick blakey,SYD,WBD
1,1.10,R18,sydney v western-bulldogs,To Get 15 or More Disposals,oliver florent,SYD,WBD
2,1.16,R18,sydney v western-bulldogs,To Get 15 or More Disposals,james rowbottom,SYD,WBD
3,1.31,R18,sydney v western-bulldogs,To Get 15 or More Disposals,justin mcinerney,SYD,WBD
4,1.32,R18,sydney v western-bulldogs,To Get 15 or More Disposals,isaac heeney,SYD,WBD
...,...,...,...,...,...,...,...
895,10.50,R18,west-coast v richmond,To Get 30 or More Disposals,trent cotchin,WCE,RCH
896,12.50,R18,west-coast v richmond,To Get 30 or More Disposals,nick vlastuin,WCE,RCH
897,21.00,R18,west-coast v richmond,To Get 30 or More Disposals,jack ross,WCE,RCH
898,26.00,R18,west-coast v richmond,To Get 30 or More Disposals,liam baker,WCE,RCH


In [35]:
output.to_csv(f'/Users/caseyh/Desktop/footyfootyfooty/sportsbet_odds/r{round_}/disposal_markets_{day}.csv', index = False)